In [1]:
import json
import numpy as np
import yfinance as yf
import random
random.seed(0)
np.random.seed(0)
from pytickersymbols import PyTickerSymbols

def is_valid_ticker(ticker):
    """Check if a ticker exists by retrieving 1 day of historical data."""
    try:
        data = yf.Ticker(ticker).history(period="1d")
        return not data.empty  # Valid if data is not empty
    except:
        return False  # Invalid ticker
    

def randomly_assign_qubits(n_qubits, stocks):
    """
    Randomly assign qubits to stocks, ensuring each stock gets at least one qubit.
    
    Args:
        n_qubits: Total number of qubits
        stocks: List of stock tickers
        
    Returns:
        qubits_per_stock: Dictionary with number of qubits per stock
    """
    n_stocks = len(stocks)
    if n_qubits < n_stocks and n_qubits != n_stocks:
        raise ValueError("Number of qubits must be at least equal to number of stocks")
    
    # Start by giving each stock one qubit
    qubits_per_stock = {stock: 1 for stock in stocks}
    
    # Randomly distribute remaining qubits
    remaining_qubits = n_qubits - n_stocks
    
    # Use random.choices with weights to distribute remaining qubits
    for _ in range(remaining_qubits):
        # Randomly select a stock to give an additional qubit
        chosen_stock = np.random.choice(stocks)
        qubits_per_stock[chosen_stock] += 1
    
    return qubits_per_stock

stock_data = PyTickerSymbols()
stocks = list(stock_data.get_dow_jones_nyc_yahoo_tickers())
stocks = [s for s in stocks if is_valid_ticker(s)]
print("Available stocks: ", len(stocks))

experiment_data = {}

for experiment_id in range(1):
    experiment_data[experiment_id] = []
    
    for n_stocks in range(4, 11):
        for max_qubits in range(n_stocks, 11):
            sampled_stocks = random.sample(stocks, n_stocks)

            start = "2015-01-01"
            end = "2025-01-01"
            data = yf.download(sampled_stocks, start=start, end=end, progress=False)
            prices_now = data["Close"].iloc[-1]

            qubits = list(range(max_qubits))

            stock_metadata = {}
            
            # Divide qubits randomly for each stock so that they sum up to max_qubits
            qubits_per_stock = randomly_assign_qubits(max_qubits, sampled_stocks)
            budget = 0
            for stock in sampled_stocks:
                ints_for_stock = list(range(1, int("1"*qubits_per_stock[stock], 2) + 1))
                random_int = np.random.choice(ints_for_stock)
                budget += prices_now[stock]*random_int

                stock_metadata[stock] = {
                    "max_int": ints_for_stock[-1],
                    "qubits": qubits_per_stock[stock],
                    "random_int": int(random_int),
                    "price": float(prices_now[stock])
                }
            print("Budget: ", budget)
            
            data_point = {}
            data_point["max_qubits"] = max_qubits
            data_point["budget"] = budget
            data_point["stock_metadata"] = stock_metadata
            data_point["stocks"] = sampled_stocks
            data_point["start"] = start
            data_point["end"] = end
            data_point["n_stocks"] = n_stocks
        
            experiment_data[experiment_id].append(data_point)

with open("experiments_data.json", "w") as f:
    json.dump(experiment_data, f, indent=4)

$GS-PK: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Available stocks:  30
YF.download() has changed argument auto_adjust default to True
Budget:  713.0215435028076
Budget:  1445.150535583496
Budget:  557.6415519714355
Budget:  1260.3635215759277
Budget:  973.2910461425781
Budget:  7726.499334335327
Budget:  2156.2599868774414
Budget:  1128.5356483459473
Budget:  1529.1049423217773
Budget:  1303.7961540222168
Budget:  1007.5248680114746
Budget:  2353.8930854797363
Budget:  1930.1630783081055
Budget:  1779.321678161621
Budget:  1475.6279335021973
Budget:  1978.2614631652832
Budget:  2910.3533477783203
Budget:  1645.4164543151855
Budget:  1444.8901290893555
Budget:  1894.990779876709
Budget:  1813.9109191894531
Budget:  2088.5116806030273
Budget:  1615.4693412780762
Budget:  1766.7822723388672
Budget:  2538.3407287597656
Budget:  1799.037254333496
Budget:  2242.7171478271484
Budget:  2089.9633293151855
